# Data Journalism Lesson 26: Finishing touches

Learn how to add the final visual flourishes to make your charts professional and presentable.

In [ ]:
import warnings
from IPython.core.interactiveshell import InteractiveShell

# Keep hold of the real method
_orig_should_run = InteractiveShell.should_run_async

# Wrap it so that any DeprecationWarning it emits is silenced
def should_run_async(self, code, *args, **kwargs):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=DeprecationWarning)
        return _orig_should_run(self, code, *args, **kwargs)

# Apply the monkey‑patch
InteractiveShell.should_run_async = should_run_async

In [ ]:
import micropip
await micropip.install('plotly')
await micropip.install('nbformat>=4.2.0') 

In [ ]:
from IPython.display import display, HTML
import pandas as pd
import math

# --- Data Loading and Initial Preparation ---
national_df = pd.read_csv("../_static/college-cost/national.csv")
exclusivencheap_df = national_df[
    (national_df['ADM_RATE'].notna()) & (national_df['ADM_RATE'] < 0.2) & 
    (national_df['COSTT4_A'].notna()) & (national_df['COSTT4_A'] < 50000)
].copy()
valid_adm_rates = national_df['ADM_RATE'].dropna()
valid_costs = national_df['COSTT4_A'].dropna()
actual_average_admit_rate = valid_adm_rates.mean() if not valid_adm_rates.empty else 0
actual_average_cost = valid_costs.mean() if not valid_costs.empty else 0

# Groundhog Predictions Data
groundhog_predictions_2024_raw = {"Early Spring": 55, "Longer Winter": 19} # R: predictions <- c("Early Spring"=55, "Longer Winter"=19)
groundhog_pred_24_iron_raw = {"Early Spring": 55, "Longer Winter": 19, "Unknown/Other": 1} # R: prediction24 <- c("Early Spring"=55, "Longer Winter"=19, 1)
groundhog_pred_23_iron_raw = {"Early Spring": 40, "Longer Winter": 35} # R: prediction23 <- c("Early Spring"=40, "Longer Winter"=35)

## The Goal

In this lesson, you'll learn how to apply professional finishing touches to your data visualizations. By the end of this tutorial, you'll understand how to enhance the readability and visual appeal of your charts through strategic use of themes, text formatting, and layout adjustments. You'll practice customizing titles, subtitles, axes, and other chart elements to create polished, publication-ready graphics. These skills will enable you to present your data journalism projects with a level of professionalism that meets industry standards.

## Why Visualize Data?

If you're a student of journalism, and you've taken your studies seriously, you've learned by now that often the most direct path to better writing is taking stuff out.

Focus on the main message. The point. The "give a shit factor" an old editor of mine used to call it. Then ruthlessly remove everything that doesn't focus directly on that. 

A graphic is no different. 

"Just as a good editor of prose ruthlessly prunes out unnecessary words, so a designer of statistical graphics should prune out ink that fails to present fresh data-information," Tufte wrote in his 1983 work The Visual Display of Quantitative Information. "Although nothing can replace a good graphical idea applied to an interesting set of numbers, editing and revision are essential to sound graphical design work as they are to writing." 

Tufte's worldview on graphics can be described as minimalist. If it wasn't absolutely essential to the graphic, remove it. He wrote an entire chapter aiming his criticism at what he called Chartjunk -- unnecessary textures, flourishes, images and other distractions that took attention away from the data. 

He summarized his worldview thusly:

> Above all else show the data. Maximize the data-ink ratio. Erase non-data-ink. Erase redundant data-ink. Revise and edit.

At the time -- when computers were hard to come by and charts were ultimately printed to be consumed -- much of Tufte made sense. Anything extraneous to a chart cost money and time. But as time moved on, criticism of Tufte's militant minimalism started to grow. 

We find ourselves in a visual age. The internet is very visual. Our social media is largely built around images and video. For graphics to break through the visual noise, information designers argue now that a bit of art, a bit of creativity, is needed. You don't break Tufte's rules -- you bend them a little.

The difference between a beginner and a professional graphic is a series of choices. They are the finishing touches that make all the differences. They'll be defined by what we include ... and what we remove.

## The Basics

For the past two tutorials, we've been circling around the last bits you need to make a graphic into something publishable. With annotations, we learned about the need for additional text like headlines and chatter, as well as fixing the axis labels so an average person can read it. We learned that every graphic needs the following to be reasonably complete:

* Headline
* Chatter
* Main body
* Annotations
* Labels
* Source line
* Credit line

Here's a graphic from The New York Times, annotated:

```{image} ../figures/nyt-graphic.png
:alt: Visual representation of different join types
:width: 600px
:align: center
```

After text came color. We want to use color sparingly to draw attention. We want to use contrast to give people a giant arrow pointing at what we want them to see. 

The last step -- the finishing touches, if you will -- for a graphic boil down to two things: The first is ditching what plotting libraries do by default. The second is using typography to create a text hierarchy that separates our text and sets a level of importance in people's minds.

Let's explore fixing up our plot's output. We'll need a graphic to work with first, so I'm going to use the scatterplot we made in the annotations chapter.

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
# Reconstruct the scatterplot from the 'annotations' chapter
fig_scatter_initial = go.Figure()

fig_scatter_initial.add_trace(go.Scatter(
    x=national_df['ADM_RATE'], y=national_df['COSTT4_A'], mode='markers', 
    marker=dict(color='grey', size=5), name='All Schools'
))
fig_scatter_initial.add_trace(go.Scatter(
    x=exclusivencheap_df['ADM_RATE'], y=exclusivencheap_df['COSTT4_A'], mode='markers+text',
    text=exclusivencheap_df['INSTNM'], textposition='top right', textfont=dict(size=9, color='darkred'),
    marker=dict(color='red', size=7), name='Exclusive & Inexpensive'
))
fig_scatter_initial.add_vline(x=actual_average_admit_rate, line_width=1, line_dash="dash", line_color="grey")
fig_scatter_initial.add_hline(y=actual_average_cost, line_width=1, line_dash="dash", line_color="grey")

fig_scatter_initial.add_annotation(x=0.87, y=85000, text="High admits, high cost", showarrow=False, font=dict(color="blue", size=10))
fig_scatter_initial.add_annotation(x=0.3, y=15000, text="Fewer admits, below average cost", showarrow=False, font=dict(color="blue", size=10))

title_text_initial = "Exclusive doesn't mean expensive"
subtitle_text_initial = "According to the federal Department of Education, there are seven schools who admit fewer than <br>20 percent of applicants but are among the most affordable."
caption_text_initial = "Source: US Department of Education | By Your Name"
full_title_initial = f"{title_text_initial}<br><sub>{subtitle_text_initial}</sub>"

fig_scatter_initial.update_layout(
    title_text=full_title_initial,
    xaxis_title="Admit rate", yaxis_title="Average annual cost",
    height=700, showlegend=True, legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    annotations=fig_scatter_initial.layout.annotations + tuple([
        dict(text=caption_text_initial,showarrow=False,xref="paper", yref="paper",x=0, y=-0.12,xanchor='left', yanchor='top',align='left')
    ]),
    margin=dict(b=100)
)
fig_scatter_initial.update_xaxes(tickformat='.0%')
fig_scatter_initial.update_yaxes(tickprefix='$', tickformat=',.0f')

fig_scatter_initial.show()

The first thing you should do with any chart that isn't a waffle chart -- more on that later -- is remove the default plot look and background by adding a pre-defined theme or customizing the layout. Plotly's default theme is `plotly`, which has a light grey background. For a cleaner look, you can use a Plotly template like `plotly_white`.

### Exercise 1: Dump the grey background

Modify the `fig_scatter_initial` by updating its layout. Set the `template` to `"plotly_white"`.

In [ ]:
fig_ex1 = go.Figure(fig_scatter_initial)

fig_ex1.update_layout(
    template="____"
)
fig_ex1.show()

Compare that to your previous chart. Easier to read, right?

We're off to a good start, but our text has no real hierarchy. We want our headline to stand out more. So let's change that. When it comes to changing text in Plotly, you do this within `update_layout`, specifically by modifying `title.font`, `xaxis.title.font`, `yaxis.title.font`, etc. [Plotly's documentation on Text and Annotations](https://plotly.com/python/text-and-annotations/) and [Layout options](https://plotly.com/python/reference/layout/) provides extensive details.

### Exercise 2: Bigger and bolder

We'll start easy. Let's make the headline bigger -- size 20 -- and bold. A common approach is `family="Arial, sans-serif", weight="bold"`.

In [ ]:
fig_ex2 = go.Figure(fig_ex1) # Copy the figure from Exercise 1

fig_ex2.update_layout(
    title_font=dict(
        size=_____, 
        family="_____",
        weight="_____"
    )
)
# Another common way to make title bold is to use HTML in the title string itself if direct font weight is tricky:
# current_title_text = fig_ex2.layout.title.text
# if not current_title_text.startswith("<b>"):
#    fig_ex2.update_layout(title_text=f"<b>{current_title_text}</b>")
fig_ex2.show()

Now let’s fix a few other issues – like the axis labels being too big, the subtitle could be smaller and let’s drop some grid lines. Let’s make the subtitle 10, the axis labels 8. Scroll down and take a look.

### Exercise 3: More text hierarchy and cleaner grid

In [ ]:
fig_ex3 = go.Figure(fig_ex2) # Copy from Exercise 2

fig_ex3.update_layout(
    xaxis_title_font_size=____,
    yaxis_title_font_size=____,
    title_font_size=20
)
fig_ex3.update_xaxes(showgrid=____)
fig_ex3.update_yaxes(showgrid=____)

fig_ex3.show()

### Exercise 4: Fixing the indent (Title Position)

Continue with `fig_ex3`. Update its layout to position the title to the left of the plot area. Set `title_x=0` and `title_xanchor='left'`.

In [ ]:
fig_ex4 = go.Figure(fig_ex3) # Copy from Exercise 3

fig_ex4.update_layout(
    title_x=____,         # Value between 0 and 1 for horizontal position
    title_xanchor='____'  # 'left', 'center', or 'right'
)
fig_ex4.show()

## Waffle charts require special attention

Frequently in my classes, students use the waffle charts library quite extensively to make graphics. This is a quick walkthrough on how to get a waffle chart into a publication ready state. 

Previously, you've experimented with `pywaffle`. We can create a similar visual using a Plotly scatter plot approach to mimic the squares.

Let's make a quick waffle chart using my most favorite silly dataset: Climate prognostications from groundhogs, other assorted animals, taxidermied animals and other objects from around the United States. You thought Punxatawney Phil was the only one? Oh no. There were 73 others in 2024 who made a prediction on early spring vs. longer winter. So what was the consensus? Let's visualize. 

To create a waffle chart in Plotly:
1.  Prepare the data: For each category, generate a list of items (e.g., 55 items for "Early Spring").
2.  Determine grid layout: Given a number of rows (e.g., `rows = 5`), calculate the number of columns needed.
3.  Calculate x, y coordinates for each square.
4.  Use `go.Scatter` with square markers (`marker_symbol='square'`).
5.  Apply labels and theme adjustments.

In [ ]:
# Helper function to generate waffle data for Plotly scatter
def get_waffle_plot_data(data_dict, num_rows, colors_dict):
    """Prepares data for a Plotly scatter-based waffle chart."""
    all_items = []
    for category, count in data_dict.items():
        all_items.extend([category] * count)
    
    total_items = len(all_items)
    if total_items == 0 or num_rows <= 0:
        return pd.DataFrame(columns=['x', 'y', 'category', 'color'])
        
    num_cols = math.ceil(total_items / num_rows)
    if num_cols == 0: # Avoid division by zero if total_items is very small and num_rows large
        num_cols = 1
        
    df_list = []
    for i, category_name in enumerate(all_items):
        row_num = i // num_cols
        col_num = i % num_cols
        df_list.append({'x': col_num, 'y': num_rows - 1 - row_num, 'category': category_name, 'color': colors_dict.get(category_name, 'grey')})
        
    return pd.DataFrame(df_list)

# Data for the first waffle chart
predictions_data_waffle1 = groundhog_predictions_2024_raw # {"Early Spring": 55, "Longer Winter": 19}
waffle1_rows = 5
waffle1_colors = {"Early Spring": "yellow", "Longer Winter": "lightblue"}

waffle_df_w1 = get_waffle_plot_data(predictions_data_waffle1, waffle1_rows, waffle1_colors)

fig_waffle1 = go.Figure()

# Plotly Express can simplify this if we have the DataFrame prepared
fig_waffle1 = px.scatter(
    waffle_df_w1, 
    x='x', 
    y='y', 
    color='category',
    color_discrete_map=waffle1_colors,
    custom_data=['category'] # For hover
)
fig_waffle1.update_traces(
    marker=dict(symbol='square', size=15), # Adjust size as needed
    hovertemplate="%{customdata[0]}<extra></extra>"
)

# Apply labs and theme adjustments
fig_waffle1.update_layout(
    title_text="Early spring it is<br><sub>The groundhogs, lobsters and other assorted predictors from around the <br>US and Canada were decisively feeling spring this year.</sub>",
    title_font_size=16,
    # title_font_family="Arial, bold", # Making title bold
    xaxis_title_text="1 square = 1 prediction",
    xaxis_title_font_size=10,
    yaxis_title_text="", # Blank Y axis title
    plot_bgcolor='white',
    paper_bgcolor='white',
    showlegend=True, # Show legend for categories
    legend_title_text='Prediction',
    height=300, # Adjust height
    width=max(500, math.ceil(sum(predictions_data_waffle1.values()) / waffle1_rows) * 20), # Adjust width based on cols
    margin=dict(t=100, b=50, l=20, r=20),
    # Aspect ratio and axis appearance for waffle effect
    yaxis_scaleanchor="x",
    yaxis_scaleratio=1,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, range=[-0.5, math.ceil(sum(predictions_data_waffle1.values())/waffle1_rows)-0.5]),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, range=[-0.5, waffle1_rows-0.5])
)
# Add caption as annotation
fig_waffle1.add_annotation(text="Source: Groundhog-Day.com | Graphic by Matt Waite",
                            showarrow=False, xref="paper", yref="paper",
                            x=0, y=-0.15, xanchor='left', yanchor='top')
fig_waffle1.show()

But what if we're using a waffle iron (multiple waffles combined)? It gets tougher. 

In Plotly, this means using `make_subplots`. Each waffle will be a trace added to a specific subplot cell. You'll apply `labs` and `theme` adjustments per subplot if needed, or globally via `update_layout`.

In [ ]:
# Data for the 'iron' waffle chart (subplots)
waffle_iron_rows = 5 # Common number of rows for each waffle
colors_pred24 = {"Early Spring": "yellow", "Longer Winter": "lightblue", "Unknown/Other": "lightgrey"}
colors_pred23 = {"Early Spring": "yellow", "Longer Winter": "lightblue"}

df_pred24 = get_waffle_plot_data(groundhog_pred_24_iron_raw, waffle_iron_rows, colors_pred24)
df_pred23 = get_waffle_plot_data(groundhog_pred_23_iron_raw, waffle_iron_rows, colors_pred23)

fig_waffle_iron = make_subplots(rows=2, cols=1, subplot_titles=(
    "Early spring it is<br><sub>The groundhogs, lobsters and other assorted predictors from around the <br>US and Canada were decisively feeling spring this year.</sub>", 
    ""
))

# Waffle for 2024 (top subplot)
for category, color in colors_pred24.items():
    cat_df = df_pred24[df_pred24['category'] == category]
    fig_waffle_iron.add_trace(go.Scatter(
        x=cat_df['x'], y=cat_df['y'], mode='markers',
        marker=dict(symbol='square', size=12, color=color),
        name=category, legendgroup='2024',
        customdata=cat_df[['category']],
        hovertemplate="%{customdata[0]}<extra></extra>"
    ), row=1, col=1)

# Waffle for 2023 (bottom subplot)
for category, color in colors_pred23.items():
    cat_df = df_pred23[df_pred23['category'] == category]
    fig_waffle_iron.add_trace(go.Scatter(
        x=cat_df['x'], y=cat_df['y'], mode='markers',
        marker=dict(symbol='square', size=12, color=color),
        name=category, legendgroup='2023',
        customdata=cat_df[['category']],
        hovertemplate="%{customdata[0]}<extra></extra>"
    ), row=2, col=1)

# Layout adjustments for the combined plot
fig_waffle_iron.update_layout(
    height=600, 
    plot_bgcolor='white', paper_bgcolor='white',
    showlegend=True, # Show one legend for all categories
    legend_tracegroupgap=180, # Space between legend groups
    title_font_size=16, # For subplot titles
    margin=dict(t=120, b=80)
)

# Update axes for each subplot to look like a waffle
num_cols_24 = math.ceil(sum(groundhog_pred_24_iron_raw.values()) / waffle_iron_rows)
num_cols_23 = math.ceil(sum(groundhog_pred_23_iron_raw.values()) / waffle_iron_rows)

fig_waffle_iron.update_xaxes(showgrid=False, zeroline=False, showticklabels=False, range=[-0.5, num_cols_24-0.5], row=1, col=1, title_text="2024", title_font_size=10)
fig_waffle_iron.update_yaxes(showgrid=False, zeroline=False, showticklabels=False, scaleanchor="x1", scaleratio=1, range=[-0.5, waffle_iron_rows-0.5], row=1, col=1)

fig_waffle_iron.update_xaxes(showgrid=False, zeroline=False, showticklabels=False, range=[-0.5, num_cols_23-0.5], row=2, col=1, title_text="2023<br>1 square = 1 prediction", title_font_size=10)
fig_waffle_iron.update_yaxes(showgrid=False, zeroline=False, showticklabels=False, scaleanchor="x2", scaleratio=1, range=[-0.5, waffle_iron_rows-0.5], row=2, col=1)

# Add overall caption
fig_waffle_iron.add_annotation(text="Source: Groundhog-Day.com | Graphic by Matt Waite",
                            showarrow=False, xref="paper", yref="paper",
                            x=0.0, y=-0.08, xanchor='left', yanchor='top')
    
# Adjust subplot title font (main title of first subplot)
fig_waffle_iron.layout.annotations[0].font.size=16 # Main title for first subplot
# fig_waffle_iron.layout.annotations[0].font.family="Arial, bold" # If bold needed

fig_waffle_iron.show()

Note: These finishing touches aren't optional. They're required of every graphic you create going forward. They are what makes professional graphics look professional. It's the careful consideration of every element on the screen and how they impact a reader's understanding of the graphic.

## The Recap

Throughout this lesson, you've mastered the art of refining your data visualizations for professional presentation. You've learned to apply clean themes, adjust text hierarchy, customize chart elements, and fine-tune layouts for both standard Plotly charts and custom-built waffle charts. Remember, these finishing touches are not just cosmetic - they're required for effectively communicating your data story to your audience. Always consider how these final adjustments can enhance the clarity and impact of your visualizations.

## Terms to Know

- **Plotly Template**: A predefined set of visual parameters in Plotly that control the overall appearance of a chart (e.g., `"plotly_white"`). Applied via `fig.update_layout(template=...)`.
- **`fig.update_layout()`**: The primary Plotly method for modifying chart layout properties like titles, fonts, axis properties, background colors, grid lines, and legend.
- **`title_font` (in `update_layout`)**: A dictionary to control title font properties (e.g., `size`, `family`, `color`).
- **`xaxis_title_font_size`, `yaxis_title_font_size`**: Specific properties to control the font size of X and Y axis titles.
- **`showminorgrid` (in `update_xaxes`/`update_yaxes`)**: Boolean to show or hide minor grid lines.
- **`title_x`, `title_xanchor`**: Properties in `update_layout` to control the horizontal position and anchor of the main chart title.
- **Waffle Chart (Plotly context)**: Often custom-built using scatter plots with square markers or heatmaps to represent parts of a whole, as Plotly lacks a dedicated waffle geometry.
- **`make_subplots()`**: A Plotly function to create figures with multiple subplots, allowing for combined visualizations like an 'iron waffle'.
- **`fig.update_xaxes()`, `fig.update_yaxes()`**: Methods to customize individual axes, including hiding gridlines, tick labels, or setting scale anchors for aspect ratios.